### Part 5: Sentiment Classification with Deep Learning using Keras
In this final notebook, we cover the implementation of advanced techniques for sentiment analysis using modern techniques in deep learning. The implementation of these techniques may often require longer training periods but can often produce better classification results if the right architecture and features are used.

This notebook covers the following:

#### Dense Layer Neural Network for Sentiment Classification
1. Implementation of Dense Layer in Keras
2. Activation and Optimization metric set up
3. Model Metric Specification

#### Recurrent Neural Networks for Sentiment Classification
1. Embedding layers with Keras
2. Recurrent Neural Network Architecture
3. Keras Model Implementation and Parameter Setting

### 0. Data Preparation
Before we implement any of the above, we need to perform the following operations on the review dataset.

1. Text Processing - Cleaning, Removing Punctuations and Numeric values
2. Build a word2vector model - Using Word vectors for feature generation
3. Averaging Word Vectors across review/text


Let's begin

In [1]:
import re
import numpy as np
import pandas as pd 
 
from keras.preprocessing.text import text_to_word_sequence
from gensim.models import Word2Vec

### 0.1. Preprocessing Text

The function below implements basic preprocessing steps to clean the text ahead of developing word vectors

In [2]:
def processText(text):
  """ Cleaning Function """
  text = re.sub('[^a-zA-Z]', ' ', text)
  text = re.sub('[0-9]+', '', text)
  text = [word for word in text_to_word_sequence(text) ]
  return " ".join(text)

In [3]:
review_data = pd.read_csv('restaurant_reviews.tsv', delimiter='\t')
review_data['clean_review'] = review_data.Review.apply( lambda x: processText(str(x)) )
review_data.head()

,Review,Liked,clean_review
0,Wow... Loved this place.,1,wow loved this place
1,Crust is not good.,0,crust is not good
2,Not tasty and the texture was just nasty.,0,not tasty and the texture was just nasty
3,Stopped by during the late May bank holiday of...,1,stopped by during the late may bank holiday of...
4,The selection on the menu was great and so wer...,1,the selection on the menu was great and so wer...


### 0.2 . Building Word2Vec Model: CBOW

The implementation below develops a CBOW word-to-vector model of vector size 500 and window size 150.

In [4]:
vector_size = 500
window_size = 150
  
corpus = [text_to_word_sequence(review) for review in review_data.clean_review.values]
cbow_model = Word2Vec( sentences= corpus, vector_size = vector_size, window = window_size, sg=0,  min_count = 2, sample=.000001 )

### 0.3. Averaging Word Vectors
The last step is to average the word vectors in the CBOW model so that we can have a single vector that summarizes each individual vector in the sentiment.

In [5]:
def avg_words_vectors(words, model, vocabulary , num_features ):
    """  """
    feature_vector = np.zeros((num_features,), dtype='float64')
    word_count = 0
    
    for word in words:
        if word in vocabulary:
            word_count += 1
            feature_vector = np.add(feature_vector, model.wv.get_vector(word))
    
    if word_count:
        feature_vector = np.divide(feature_vector, word_count)
    
    return feature_vector

def word_vectorizer(corpus, model, num_features):
    """ Average Word Vectors """
    vocabulary = list(model.wv.index_to_key)
    
    features = [ avg_words_vectors(sentence, model, vocabulary, num_features) for sentence in corpus ]   
    return np.array(features)

In [6]:
text_features = word_vectorizer(corpus=corpus, model=cbow_model, num_features=500)
text_features.shape

(1000, 500)

### 1. Classification with Dense Neural Network
In this example, we will implement a simple fully connected Neural Network for classification. The neural network will have 2-3 Dense/Fully connected layers with drop-out layers to avoid overfitting. The output layer will be a binary output.

Initial parameter for the model:

1. batch_size = 50
2. Training Epochs = 20
3. Input_size/Vector Size = 500

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.utils import to_categorical


# Dense Layer Architecture
fully_connected_nn = Sequential()
fully_connected_nn.add(Dense(200, activation='relu', input_shape=(vector_size, )))
fully_connected_nn.add(Dropout(.5))
fully_connected_nn.add(Dense(200, activation='relu'))
fully_connected_nn.add(Dropout(.5))
fully_connected_nn.add(Dense(200, activation='relu'))
fully_connected_nn.add(Dropout(.5))
fully_connected_nn.add(Dense(2, activation='softmax'))

In [8]:
fully_connected_nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               100200    
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 200)               40200     
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 200)               40200     
                                                                 
 dropout_2 (Dropout)         (None, 200)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 4

The next step is to compile the model before training it on the dataset.

In [9]:
fully_connected_nn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### 1.1. Training/Fitting the Model

To train the model, we pass on features and targets, train and test split (as test data percentage), epochs for training, and batch sizes.



In [10]:
target = to_categorical(review_data.Liked)

fully_connected_nn.fit(text_features, target , validation_split=.3, shuffle=True, epochs=50, batch_size=150, verbose=1)


Epoch 1/50
5/5 [==============================] - 4s 351ms/step - loss: 0.6918 - accuracy: 0.5471 - val_loss: 0.7025 - val_accuracy: 0.3600
Epoch 2/50
5/5 [==============================] - 0s 65ms/step - loss: 0.6890 - accuracy: 0.5600 - val_loss: 0.7157 - val_accuracy: 0.3600
Epoch 3/50
5/5 [==============================] - 0s 59ms/step - loss: 0.6873 - accuracy: 0.5600 - val_loss: 0.7262 - val_accuracy: 0.3600
Epoch 4/50
5/5 [==============================] - 0s 57ms/step - loss: 0.6857 - accuracy: 0.5600 - val_loss: 0.7351 - val_accuracy: 0.3600
Epoch 5/50
5/5 [==============================] - 0s 41ms/step - loss: 0.6853 - accuracy: 0.5600 - val_loss: 0.7408 - val_accuracy: 0.3600
Epoch 6/50
5/5 [==============================] - 0s 55ms/step - loss: 0.6872 - accuracy: 0.5600 - val_loss: 0.7393 - val_accuracy: 0.3600
Epoch 7/50
5/5 [==============================] - 0s 67ms/step - loss: 0.6851 - accuracy: 0.5600 - val_loss: 0.7410 - val_accuracy: 0.3600
Epoch 8/50
5/5 [==========

### 1.2. Model Performance and Improvements
The trained model has an 89% accuracy on the train set and 76% accuracy on the test set which is very good performance for a simple dense model.

<br>
Notice that the validation loss is increasing as the training loss is decreasing which suggests overfitting. To improve performance, we can introduce more regularizers and change the architecture. For our purposes, this is sufficient. We move on to using RNNs.

### 2. Embedding Layers and Recurrent Neural Networks
Word Embeddings are a little different from the averaged word vector features we used in our deep layer neural network because they use word indexing in place for the words in the corpus and assign an indeces to the words to represent their presence or lack of in a particular document.

<br>
Let's demonstrate this in practice. We can first count the frequency of the words below and use the variable to determine the number of unique words in the corpus

In [11]:
from collections import Counter

token_counter = Counter([token for review in corpus for token in review])

Let's use the token counter to create a dictionary that indexes all the words to some unique value.

In [12]:
vocab_map = { item[0]:index+1 for index,item in enumerate(dict(token_counter).items()) } # Index all the words starting at 1
max_index = np.max(list(vocab_map.values()))   

Notice that we may have instances when loading new text that the words are not in the vocabulary map. In the code below, we add two things: a padding index value matched to index 0 and not found index as the last words to catch those two scenarios.

In [13]:
vocab_map['PAD_INDEX'] = 0
vocab_map['NOT_FOUND_INDEX'] = max_index + 1
vocab_size = len(vocab_map)

We also need to obtain the maximum length of the review. We use the following list comprehension to return the maximum review.

In [14]:
max_len = np.max([len(review) for review in corpus])
max_len

32

### 2.1. Padding Text Sequences

We need to pad the text sequences to make sure that all reviews have the same input length. To do this, we use the keras padding method.

In [15]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

padding_reviews = [[vocab_map[token] for token in review] for review in corpus]
input_features = pad_sequences(padding_reviews, max_len)

input_features

array([[   0,    0,    0, ...,    2,    3,    4],
       [   0,    0,    0, ...,    6,    7,    8],
       [   0,    0,    0, ...,   13,   14,   15],
       ...,
       [   0,    0,    0, ...,    7,   76,   77],
       [   0,    0,    0, ...,  516,  512,   63],
       [   0,    0,    0, ..., 1323,   11,  528]], dtype=int32)

###  2.2. Building a Recurrent Neural Network

The RNN will be a relatively simple network with one LSTM layer, an Embedding layer, and a dropout layer for regularization. See the architecture below.

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Flatten
from keras.layers import LSTM

Embedding_dim = 128
LSTM_DIM = 64

In [17]:
rnn_model = Sequential()
rnn_model.add(Embedding(input_dim=vocab_size, output_dim=Embedding_dim, input_length=max_len))
rnn_model.add(Dropout(.2))
rnn_model.add(LSTM(LSTM_DIM, dropout=.2, recurrent_dropout=.2))
rnn_model.add(Dense(2, activation='softmax'))

rnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### 2.3. Fittting the RNN Classifier
In fitting our RNN we will use the following hyperparameters:

1. batch_size = 100
2. Epochs = 20

In [18]:
rnn_model.fit(input_features, target, epochs=20, batch_size=100, shuffle=True, validation_split=.3, verbose=1)

Epoch 1/20
7/7 [==============================] - 5s 160ms/step - loss: 0.6890 - accuracy: 0.5500 - val_loss: 0.7065 - val_accuracy: 0.3600
Epoch 2/20
7/7 [==============================] - 1s 101ms/step - loss: 0.6754 - accuracy: 0.5600 - val_loss: 0.7368 - val_accuracy: 0.3600
Epoch 3/20
7/7 [==============================] - 1s 158ms/step - loss: 0.6639 - accuracy: 0.5614 - val_loss: 0.7440 - val_accuracy: 0.3600
Epoch 4/20
7/7 [==============================] - 1s 188ms/step - loss: 0.6356 - accuracy: 0.6171 - val_loss: 0.7030 - val_accuracy: 0.4033
Epoch 5/20
7/7 [==============================] - 1s 150ms/step - loss: 0.5927 - accuracy: 0.7171 - val_loss: 0.7067 - val_accuracy: 0.4467
Epoch 6/20
7/7 [==============================] - 1s 104ms/step - loss: 0.5086 - accuracy: 0.7514 - val_loss: 0.6620 - val_accuracy: 0.5767
Epoch 7/20
7/7 [==============================] - 1s 101ms/step - loss: 0.3968 - accuracy: 0.8586 - val_loss: 0.6379 - val_accuracy: 0.6833
Epoch 8/20
7/7 [====

We see that over time the accuracy increases in the training set to 99% but the validation loss decreases and then increases indicating overfitting. It is also important to note that the dataset we are using is rather small with only about 1000 textual reviews for which neural networks can easily overfit.



### Predicting Sentiment of New Reviews

We have built our model and wish to deploy it in new reviews to see the performance. We will need to preprocess the new input in the same way we preprocessed the training dataset. Let's see this in action below.

In [19]:
new_text = "the food was great!"

new_text = [[vocab_map[token] for token in text_to_word_sequence(new_text)]]
padded_text_input = pad_sequences(new_text, max_len)
padded_text_input

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,  11, 124,  13,  31]], dtype=int32)

In [20]:
rnn_model.predict(padded_text_input)

1/1 [==============================] - 1s 516ms/step


array([[0.01302621, 0.98697376]], dtype=float32)

Notice that our model has predicted 98.7% probability of a positive review and abour 1.3% probability that our sentiment is a negative review. Not bad for test prediction.

<br>

### Conclusion

In the series, we covered a number of NLP techniques from traditional feature extraction techniques like TFIDF and Bag of Words to modern techniques like Word2Vect and Embeddings. We have learned how to use word embedding and word vectors to build classification and sentiment analysis models. Try using the above model architectures and techniques to build a model for the Amazon product reviews data and set the results. Deep Learning techniques offer a lot of opportunities to improve model performance with additional layers, regularizers, and hyperparameter changes!